<a href="https://colab.research.google.com/github/y4c6/master_thesis/blob/main/post_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

無法分辨上一個到底怎麼了。普通的斷線了。

In [2]:
from google.colab import drive
# directory
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import json

In [ ]:
data_stem = pd.read_pickle("/content/gdrive/MyDrive/Thesis_Data&Result/EJMR_data_stem_ao2_a.pkl")
data_stem.rename(columns = {'y':'asian'}, inplace = True)
print(data_stem.shape)
data_stem.head(3)

(23337, 10001)


,aa,aacsb,ab,abandon,abbott,abbrevi,abc,abd,abe,abel,...,zone,zoo,zoom,zoomer,zu,zuckerberg,zucman,zuel,zurich,asian
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

train_stem, test_stem = train_test_split(data_stem, test_size=0.5,  random_state = 2510, shuffle=True)

In [ ]:
print(sum(data_stem.columns == train_stem.columns))
print(sum(data_stem.columns == test_stem.columns))

10001
10001


In [ ]:
del data_stem

In [4]:
data_lemma = pd.read_pickle("/content/gdrive/MyDrive/Thesis_Data&Result/EJMR_data_lemma_ao2_a.pkl")
data_lemma.rename(columns = {'y':'asian'})#, inplace = True)
print(data_lemma.shape)
data_lemma'shape???'.head(3) 

(23337, 10000)


,aa,aacsb,ab,abandon,abandoned,abandoning,abc,abd,abe,ability,...,zip,zju,zombie,zone,zoning,zoom,zu,zuckerberg,zucman,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data_lemma.columns

Index(['aa', 'aacsb', 'ab', 'abandon', 'abandoned', 'abandoning', 'abc', 'abd',
       'abe', 'ability',
       ...
       'zip', 'zju', 'zombie', 'zone', 'zoning', 'zoom', 'zu', 'zuckerberg',
       'zucman', 'zurich'],
      dtype='object', length=10000)

In [103]:
from sklearn.model_selection import train_test_split

train_lemma, test_lemma = train_test_split(data_lemma, test_size=0.5,  random_state = 2510, shuffle=True)

In [104]:
print(sum(data_lemma.columns == train_lemma.columns))
print(sum(data_lemma.columns == test_lemma.columns))

10000
10000


In [16]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn.linear_model import LinearRegression

class postLasso:
    def __init__(self, random_state=None, target=None, set1=None, set2=None, token = {'stem', 'lemma'}):
        self.random_state = random_state
        self.token = token  # {stem, lemma}
        self.set1 = set1
        self.set2 = set2
        self.target = target #
        #self.treatment = treatment
        self.selectX = None # param passthrough from lasso to ols
        self.alpha = None # param passthrough from lassocv to lasso
        self.lassocv = None
        self.lasso = None
        self.ols = None

    def lasso_fit(self, target):
        # Step 1: LassoCV for feature selection
        self.lasso_cv = LassoCV(random_state=self.random_state, cv=5, tol=0.1)

        self.lasso_cv.fit(self.set1.loc[:, self.set1.columns != self.target],
                 self.set1[self.target])

        # Step 2: Lasso with alpha from LassoCV
        self.alpha = self.lasso_cv.alpha_
        self.lasso = Lasso(alpha=self.alpha)
        self.lasso.fit(self.set1.loc[:, self.set1.columns != self.target],
                 self.set1[self.target])
        self.selectX = list(self.set1.keys()[self.set1.columns != self.target][self.lasso.coef_!=0])

    def get_selectX(self):
        return self.selectX

    def post_lasso_fit(self, target, treatment):
        # Step 3: OLS with selected features
        #self.selectX = list(self.set1.keys()[self.set1.columns != self.target][self.lasso.coef_!=0])
        
        self.treatment = treatment
        self.ols = LinearRegression()
        self.ols.fit(self.set2[ self.treatment + self.selectX ], 
              self.set2[ self.target ])

    # def predict(self, X):
    #     X_selected = X[:, self.selected_features]
    #     return self.ols.predict(X_selected)

    def get_ols_model(self):
        return self.ols




In [ ]:
##########stem##########
## behavior = ['hardwork', 'cheat', 'encourag']  # stem_asian_behavior.csv
## appearance = ['golddigger', 'ugli', 'fivefooteight', 'undesir', 'despis']      # stem_asian_appearance.csv
## intellengence = ['braindamag', 'brainwash', 'creativ', 
#                  'submoron', 'stjupid', 'stpid', 'mediocr', 'talent'] # stem_asian_intellengence.csv
## trash = ['rubbish', 'trash', 'rag']                          # stem_asian_trash.csv
# fword = ['fuaak', 'phuyk', 'flopper']                          # none of these in colname
# sword = ['shit', 'shiet', 'shi', 'stter']                       # stem_asian_shiit.csv
# adjs = ['shame','weak','solid','stronger','good','strongest','great','bad','impress','weaker','terribl','clear','well']  # stem_asian_adjs.csv

In [51]:
y_name = 'asian'

In [96]:
adjs = ['shame','weak','solid','stronger','good','strongest','great','bad','impress','weaker','terribl','clear','well']

In [97]:
for w in adjs:
  print(w, ( w in list(train_stem.columns)) & ( w in list(test_stem.columns) ) ) #w in list(data_stem.columns)

shame True
weak True
solid True
stronger True
good True
strongest True
great True
bad True
impress True
weaker True
terribl True
clear True
well True


----

In [37]:
asian_model = postLasso(random_state=4205, target='asian', set1=train_stem, set2=test_stem, token = 'stem')
asian_model.lasso_fit(target='asian')

In [48]:
selectX = asian_model.get_selectX()

In [70]:
print(len(selectX))
print(selectX)

35
['adb', 'american', 'chines', 'countri', 'crime', 'east', 'econom', 'happi', 'hk', 'immigr', 'indian', 'interview', 'japanes', 'job', 'journal', 'karl', 'kimchi', 'ntu', 'one', 'paper', 'read', 'sat', 'singapor', 'south', 'student', 'taiwanes', 'tax', 'test', 'time', 'tokyo', 'use', 'white', 'women', 'work', 'year']


_______

In [98]:
D_name = adjs#[ sword[i] for i in [0,1,2] ]

In [99]:
asian_model.post_lasso_fit(target='asian', treatment=D_name)
ols2 = asian_model.get_ols_model()
ols2.coef_

array([-6.58325181e-03,  6.39454026e-03,  2.81533105e-03,  2.98115768e-03,
       -2.59498396e-03, -1.65424620e-02, -3.45280320e-03,  8.05141389e-03,
        8.56445869e-03, -1.21491876e-03,  2.85395730e-03, -3.00352197e-03,
        1.21242578e-04,  3.44606797e-03,  5.31086215e-03, -4.91665593e-03,
        8.52719625e-04,  1.53951332e-02,  2.24673801e-02, -2.19169139e-03,
       -1.18896400e-02,  1.29871177e-03,  3.72818654e-03, -1.75467295e-03,
       -2.50118389e-04,  2.36906552e-02, -3.50087960e-04, -1.52676007e-04,
       -2.15528828e-05, -8.88391014e-03,  9.76475464e-02,  3.15014214e-04,
       -6.54969610e-04, -5.20623626e-03, -3.40393196e-03,  4.56492650e-03,
        2.75834413e-02,  6.11860064e-04,  8.81477929e-03, -3.24833372e-03,
        1.10897892e-03, -2.11952406e-03,  6.98190266e-02, -1.62445374e-03,
        1.08434058e-02,  3.06996268e-04, -5.75755661e-04, -6.47143103e-04])

In [100]:
pd.DataFrame({'word': D_name + list(lasso_X),
      'coef': ols2.coef_}).to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/stem_asian_adjs.csv')

In [ ]:
##########lemma##########

In [ ]:
# manner = ['rude', 'dickhad', 'cocking']
# appearance = ['obese', 'fivefooteight', 'golddigger']
# intellegence = ['brainwashed', 'overratedbogus',
#  'mediocre', 'submoron', 'stjupid', 'dorkness']
# fword = ['fawk', 'fleckistan',
#  'fuaak', 'fawhk', 'fk', 'mf']
# sword = ['sheethole',
#  'shit', 'shiiiit', 'shtholes', 'shiat'] 
# trash = ['dvmpster',
#  'trxsh', 'rubbish', 'rag', 'trash']
# adjs = ['weak', 'weaker', 'weakest',
# 'bad', 'well', 'bright', 'shamed', 'shameful', 'unimpressive',
# 'disgiusting', 'disappointing', 'fierce', 'terrible', 'embarrassing']

In [ ]:
asian_model = postLasso(random_state=4205, target='asian', set1=train_lemma, set2=test_lemma, token = 'lemma')
asian_model.lasso_fit(target='lemma')

# old things

In [ ]:
lasso_X = post_lasso(train_stem, test_stem, y_name=y_name, random_state=5222, token='stem')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.277e+02, tolerance: 8.803e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.087e+02, tolerance: 8.803e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.078e+02, tolerance: 8.803e

Number of selected covariates: 35
['adb', 'american', 'chines', 'countri', 'crime', 'east', 'econom', 'happi', 'hk', 'immigr', 'indian', 'interview', 'japanes', 'job', 'journal', 'karl', 'kimchi', 'ntu', 'one', 'paper', 'read', 'sat', 'singapor', 'south', 'student', 'taiwanes', 'tax', 'test', 'time', 'tokyo', 'use', 'white', 'women', 'work', 'year']
stem_asian_select.csv is saved


In [18]:
## 3rd step: post-Lasso OLS
post_OLS = LinearRegression()\
       .fit(X = test_stem[ D_name + lasso_X ], 
         y = test_stem[y_name])

In [28]:
pd.DataFrame({'word': D_name + list(lasso_X),
      'coef': post_OLS.coef_}).to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/stem_ugli.csv')

In [30]:
import statsmodels.formula.api as smf
import statsmodels.regression.linear_model as sm

model = sm.OLS(endo = test_stem[ D_name + lasso_X ], 
         Y = test_stem[y_name])
results = model.fit()

results.t_test([0,1])

TypeError: ignored

In [ ]:
stem1.alpha_

0.00464099887136029

In [ ]:
type(stem_OLS)

sklearn.linear_model._base.LinearRegression

In [ ]:
stem_OLS.coef_

array([ 1.71435943e-02, -1.89833862e-02, -2.05683850e-02,  1.85070622e-02,
        2.36500914e-02, -2.69382025e-02,  2.59145359e-02,  2.50187745e-03,
        6.80545956e-04,  1.20540786e-02,  5.07507769e-03, -2.59359615e-02,
        1.28586588e-02, -2.06937857e-03, -1.12614795e-02,  1.01617274e-02,
        1.72084845e-02,  2.03867928e-02, -2.05444754e-03, -4.31643670e-03,
        1.55042398e-02,  5.58457203e-03,  1.53032062e-01, -3.40953463e-03,
        3.67470103e-03, -4.53978972e-04, -1.77311771e-02,  1.79904537e-02,
        5.24321230e-03, -5.46492782e-03, -9.89434821e-03, -2.06165428e-02,
        2.91007044e-03, -1.11325605e-02, -4.28180789e-03,  2.38927733e-02,
        2.19534330e-02,  2.16066219e-02, -1.26822126e-02,  1.18640913e-01,
       -2.15679706e-02, -2.12277806e-02,  3.25291959e-02, -1.03275325e-02,
       -5.93126885e-02,  6.38886141e-03, -2.00513259e-02,  1.31278223e-03,
        3.90647405e-02,  2.27758026e-03, -3.65731948e-02,  1.90559341e-02,
       -8.47323383e-03,  

In [ ]:
pd.DataFrame({'word': [D_name] + list(stem2_select),
      'coef': stem_OLS.coef_}).to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/stem_stupid.csv')

In [ ]:
y_name

'stupid'

In [ ]:
# from sklearn.linear_model import LassoCV # determine alpha_ (lambda in R)
# from sklearn.linear_model import Lasso # select covariates
# from sklearn.linear_model import LinearRegression # post-lasso

# def post_lasso(set1, set2, y_name:str, random_state:int, token={'stem', 'lemma'}):
#   '''
#   y_name: independent vars.
#   D_name: treatment var(s). #D_name:list
#   set1: (pd.df) random half of the dataframe
#   set2: (pd.df) the other random half of the dataframe
#   random_state: (int) for LassoCV(), may be different to the one in train_test_split
#   '''

#   # 如果一篇主題討論文中有出現關鍵字，那就算一(不論出現次數)
#   set1.loc[set1[y_name] != 0, y_name] = 1

#   ## 1st step: lassoCV 
#   lasso1 = LassoCV(cv=5, random_state = random_state, tol = 0.1)\
#      .fit( X = set1.loc[:, set1.columns != y_name],
#          y = set1[y_name])
  
#   ## 2st step: lasso
#   lasso2 = Lasso( alpha = lasso1.alpha_)\
#      .fit( X = set1.loc[:, set1.columns != y_name],
#          y = set1[y_name])
  
#   print('Number of selected covariates:',len(lasso2.coef_[lasso2.coef_!=0]))

#   lasso2_select = list(set1.keys()[set1.columns != y_name][lasso2.coef_!=0]) # remove y_name
#   print(lasso2_select)

#   pd.DataFrame({'word':list(lasso2_select),
#          'coef': lasso1.coef_[lasso2.coef_!=0]}).to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/' +token+ '_' +y_name+ '_select_test.csv')
#   print(f'{token}_{y_name}_select.csv is saved')


#   ## 3rd step: post-Lasso OLS: writen outside
#   '''
#   post_OLS = LinearRegression()\
#        .fit(X = set2[ D_name + lasso_X ], 
#          y = set2[y_name])
#   can't return this regression model
#   ''' 
#   return lasso2_select

